In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "yours"

import argparse
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

class SharedMemory:
    def __init__(self) -> None:
        self.memory = []

    def store(self, item):
        self.memory.append(item)

    def get_memory(self):
        return self.memory
    
class Agent:
    def __init__(
        self,
        system_message: BaseMessage,
        model: ChatOpenAI,
        name: str,
        shared_memory: SharedMemory
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.name = name
        self.shared_memory = shared_memory
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(self, input_message: HumanMessage) -> AIMessage:
        messages = self.update_messages(input_message)
        output_message = self.model(messages)
        self.update_messages(output_message)
        self.shared_memory.store(output_message)
        return output_message


class Simulation:
    def __init__(self, agents: List[Agent]) -> None:
        self.agents = agents

    
    
    def run_simulation(self, chat_turn_limit: int = 30) -> None:
        self.reset_agents()
        msg = self.agents[0].system_message
        n = 0
        while n < chat_turn_limit:
            n += 1
            for agent in self.agents:
                msg = HumanMessage(content=agent.step(msg).content)
                print(f"{agent.name}: {msg.content}\n")

    def reset_agents(self) -> None:
        for agent in self.agents:
            agent.reset()


def create_agents(task: str, temperature: float = 0.2) -> List[Agent]:
    shared_memory = SharedMemory()
    
    teacher_system_message = SystemMessage(
        content=f"""[ROLE: Teacher]
        I am the Teacher, and my role is to guide the Student in understanding the concept of 
        {task}. I will be providing explanations and answering any questions the Student might have. 
        I will not ask questions as if I am the Student. 
        Our conversation topic is {task}, and I am a very smart math teacher.
        I never have questions about {task} since you are the expert on it.
        I am only concerned if the student understands {task}.
        I am never answering as observer or student. I am only the teacher.
        """
    )

    student_system_message = SystemMessage(
        content=f"""[ROLE: Student]
        I am the Student, and my role is to learn from the Teacher about 
        {task}. I will be asking questions and trying to understand the concept better. 
        I will not provide explanations as if I am the Teacher. 
        Our conversation topic is {task}, and I am a student who has problems 
        understanding simple arithmetic.
        I never start explaining to the teacher or observer about the {task} since I do not know anything about it 
        beside of what the teacher explains to me.
        I am never answering as teacher or observer. I am only the student.
        """
    )
    
    observer_system_message = SystemMessage(
        content=f"""[ROLE: Observer]
        As an observer, my role is to monitor the conversation between the teacher and the student.
        I will only participate when I have something meaningful to add or to help guide the conversation.
        The topic of our conversation is: {task}.
        I never explain anything to the teacher but just add general explanations if needed.
        I am never answering as teacher or student. I am only the observer.
        """
    )

    return [
        Agent(teacher_system_message, ChatOpenAI(temperature=temperature,model_name="gpt-4-0314"), "Teacher",shared_memory), #defaul model if not specific 3.5 turbo
        Agent(student_system_message, ChatOpenAI(temperature=temperature,model_name="gpt-4-0314"), "Student",shared_memory),
        Agent(observer_system_message, ChatOpenAI(temperature=temperature,model_name="gpt-4-0314"), "Observer",shared_memory),
    ]


In [ ]:
def main(task, temperature, chat_turn_limit) -> None:
    agents = create_agents(task=task, temperature=temperature)
    simulation = Simulation(agents)
    simulation.run_simulation(chat_turn_limit=chat_turn_limit)

In [ ]:
#task = "Make the student understand why 2 + 2 equals 4"
task = "Make the student understand why red is red"
temperature = 0.2
chat_turn_limit = 15

main(task, temperature, chat_turn_limit)
